In [6]:
import tensorflow
 
conv_base = tensorflow.keras.applications.VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3)
                  )

In [7]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [9]:

model = tensorflow.keras.models.Sequential()


model.add(conv_base)


model.add(tensorflow.keras.layers.Flatten())


model.add(tensorflow.keras.layers.Dense(256, activation='relu'))
model.add(tensorflow.keras.layers.Dense(6, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer=tensorflow.keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=['acc'])


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 6)                 1542      
                                                                 
Total params: 21,139,014
Trainable params: 13,503,750
Non-trainable params: 7,635,264
_________________________________________________________________


In [10]:

train_dir = 'Skin_Conditions/TRAIN'
validation_dir = 'Skin_Conditions/VALIDATION'
test_dir = 'Skin_Conditions/TEST'

In [11]:

train_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255, 
      rotation_range=40, 
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest'
      )

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=20,
        )


validation_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        )

Found 1800 images belonging to 6 classes.
Found 114 images belonging to 6 classes.


In [12]:

history = model.fit(
      train_generator,
      steps_per_epoch = train_generator.samples // train_generator.batch_size,
      epochs=50,
      validation_data=validation_generator,
      validation_steps = validation_generator.samples // validation_generator.batch_size)


model.save('trained_tf_model.h5')

Epoch 1/50
90/90 [==============================] - 174s 2s/step - loss: 0.4236 - acc: 0.3628 - val_loss: 0.3767 - val_acc: 0.3900
Epoch 2/50
90/90 [==============================] - 160s 2s/step - loss: 0.3452 - acc: 0.5350 - val_loss: 0.3396 - val_acc: 0.5200
Epoch 3/50
90/90 [==============================] - 166s 2s/step - loss: 0.3061 - acc: 0.6033 - val_loss: 0.2829 - val_acc: 0.6100
Epoch 4/50
90/90 [==============================] - 170s 2s/step - loss: 0.2788 - acc: 0.6467 - val_loss: 0.2663 - val_acc: 0.6200
Epoch 5/50
90/90 [==============================] - 170s 2s/step - loss: 0.2569 - acc: 0.6706 - val_loss: 0.2414 - val_acc: 0.7000
Epoch 6/50
90/90 [==============================] - 166s 2s/step - loss: 0.2384 - acc: 0.7144 - val_loss: 0.2840 - val_acc: 0.6300
Epoch 7/50
90/90 [==============================] - 168s 2s/step - loss: 0.2210 - acc: 0.7372 - val_loss: 0.2520 - val_acc: 0.6800
Epoch 8/50
90/90 [==============================] - 168s 2s/step - loss: 0.2184 - a

In [13]:

test_datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255
        )

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=20,
        )


test_loss, test_acc = model.evaluate(test_generator, steps=20)
print('test acc:', test_acc)

Found 480 images belonging to 6 classes.
20/20 [==============================] - 25s 1s/step - loss: 0.1001 - acc: 0.8900
test acc: 0.8899999856948853
